# E-BC coupling for synthetic library of topic 11 cross species - Novaseq 20190405

- R1: contains enhancer
- R2: contains BC


In [ ]:
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405
mkdir 00.RAW
rsync -a /staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20190405/Demultiplexed/CH1__4856bd__CheqSeq_OLS_Liesbeth_enhancer-barcode_S1_R1_001.fastq.gz r10n1:/user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405/00.RAW
rsync -a /staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20190405/Demultiplexed/CH1__4856bd__CheqSeq_OLS_Liesbeth_enhancer-barcode_S1_R2_001.fastq.gz r10n1:/user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405/00.RAW

## Fastqc on all raw reads

In [ ]:
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405
mkdir 01.FASTQC
module load FastQC/0.11.7
fastqc -t 1 00.RAW/CH1__4856bd__CheqSeq_OLS_Liesbeth_enhancer-barcode_S1_R1_001.fastq.gz -o 01.FASTQC
fastqc -t 1 00.RAW/CH1__4856bd__CheqSeq_OLS_Liesbeth_enhancer-barcode_S1_R2_001.fastq.gz -o 01.FASTQC

# 1) Read 1: enhancer

## 1.0) Filter for sequencing quality

In [ ]:
# at least 80% of the NT (-u 20) must have a quality higher then 20 
module load fastp/20180522-foss-2014a
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405
mkdir 02.TRIMMED_FQ
fastp --disable_adapter_trimming -q 20 -u 20 -i 00.RAW/CH1__4856bd__CheqSeq_OLS_Liesbeth_enhancer-barcode_S1_R1_001.fastq.gz -o 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20.fastq.gz

## 1.1) Cutadapt the 5' adaptor

- At the 5’ end we have the adapter with 0-5 stagger before it.
- As the adaptor is always the same and at the 5’, we will use a anchored 5’ adapter to cut with cutadapt (here the adaptor will be considered as a whole and really only at the 5’ end).
- Cuadtapt is robust to errors of 10%, so it will handle the staggers and will cut the adaptors with the staggers off. (so we do not need to cut with the 6 different types of adapters when including the stagger to them)
    -> so we can just cut with the 23 bp enhancer: TGCAGGTGCCAGAACATTTCTCT



In [ ]:
# cut 5' adaptor of read 1 using the 23bp enhancer (without staggers)
# write only the reads that are trimmed, so that contained the adaptor
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405
module load cutadapt/20180522-foss-2014a-Python-2.7.6
cutadapt -g TGCAGGTGCCAGAACATTTCTCT --trimmed-only \
--output 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5.fastq.gz \
02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20.fastq.gz

Finished in 214.58 s (21 us/read; 2.87 M reads/minute).                                                                                                                                                   
                                                                                                                                                                                                          
=== Summary ===                                                                                                                                                                                           
                                                                                                                                                                                                          
Total reads processed:              10,263,235                                                                                                                                                            
Reads with adapters:                10,230,160 (99.7%)                                                                                                                                                    
Reads written (passing filters):    10,230,160 (99.7%)                                                                                                                                                    
                                                                                                                                                                                                          
Total basepairs processed: 2,565,808,750 bp                                                                                                                                                               
Total written (filtered):  2,298,783,774 bp (89.6%) 


## 1.2) Cutadapt the 3' adapter
- First we will try to find the empyt plasmids by doing cutadapt with 2x20bp of the empty vector that is not in the reads with enhancers (so further then those first 35 bp) and demand double trimming; and write the rest (not trimmed, so not empty ones) to a file ; and we will demand a minimum overlap of 5
    - We will use 20 bp somewhere at middle/beginning for 5': CGTTCGTCCTCAGTCGCGAT
    - for 3': ACTCGAGCCGAGCAGGCGCG
- The rest we trim with a small part of the vector that is normally at the 3' end: GCCTAACTGGCCGGTACCTGAGCTCCCTAGGG (32bp, can be longer or shorter; it is normally ~36bp but took here a bit less so that we have still some bp at the end left as we demand a anchored 3' adapter)

__1.2.1) Filter out empty vector__

In [ ]:
## Find empty plasmids:
    # Use linked and required adapters:
        # you can instead use -g to specify a linked adapter. In that case, both adapters are required (even if they are not anchored). 
        # - We will use 20 bp somewhere at middle/beginning for 5': CGTTCGTCCTCAGTCGCGAT
        # - for 3': ACTCGAGCCGAGCAGGCGCG
    # write the trimmed ones to an output ( = empty vector) ; the untrimmed also to an output (= vector with enhancer)
    # Minimum overlap of 5 is required --overlap

cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405
module load cutadapt/20180522-foss-2014a-Python-2.7.6
cutadapt -g CGTTCGTCCTCAGTCGCGAT...ACTCGAGCCGAGCAGGCGCG \
--overlap 5 \
--output 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5_emptyVector.fastq.gz \
--untrimmed-output 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5_full.fastq.gz  \
02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5.fastq.gz

__1.2.2) Remove 3' adapter from the ones that are not empty__

Remove the vector part at the end, to keep only the enhancer

- The rest we trim with a small part of the vector that is normally at the 3' end: GCCTAACTGGCCGGTACCTGAGCTCCCTAG (30bp, can be longer or shorter; it is normally ~36bp but took here a bit less so that we have still some bp at the end left as we demand a anchored 3' adapter)

In [ ]:
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405
module load cutadapt/20180522-foss-2014a-Python-2.7.6
cutadapt -a GCCTAACTGGCCGGTACCTGAGCTCCCTAG --trimmed-only \
--output 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full.fastq.gz \
02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5_full.fastq.gz

## 1.3) Mapping of enhancer

No we have a file with only the enhancer (~190 bp; 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full.fastq.gz)

We will map this using Minimap to our custom genome of the 14,984 designed enhancers

In [ ]:
# Map trimmed R1 (only enhancer) to custom genome with Minimap
    # this sam file still contains secondary alignemnets (flag (column 2) of 256) and bad qualtiy reads (column 5, mapping quality)
module load Minimap2/2.11-foss-2014a
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405
minimap2 -a /ddn1/vol1/staging/leuven/stg_00002/lcb/lminn/Melanoma-species/Topic11_liftover/Cloning_fastas/Total-sequence-to-clone_Liesbeth_withoutAdaptors_NoLineBreaks.fa \
02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full.fastq.gz \
> 03.MAPPING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sam

In [ ]:
# Convert to bam file and filter for mapping quality Q4
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405
module load SAMtools

samtools view -bS 03.MAPPING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sam > 03.MAPPING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.bam

samtools sort -o 03.MAPPING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.bam 03.MAPPING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.bam

samtools view -b -q 4 -@ 2 -o 03.MAPPING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4.bam 03.MAPPING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.bam

# This file is a bma file format with in column 1) ID 3) coord of the orignal desgined enahncer where it mapped to 10) the real sequenced ~190bp

In [ ]:
# Plot distribution how many reads map to which designed enhancers
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405/03.MAPPING
samtools view  CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4.bam| awk '{print $3}' |uniq -c > MappedToRegionDistribution.txt

# R
module load R/3.4.1-foss-2014a-noX
module load HDF5/1.8.12-foss-2014a-serial
R
.libPaths("/user/leuven/317/vsc31703/stg_00002/R/x86_64-pc-linux-gnu-library/3.4")


counts <- read.table("/user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405/03.MAPPING/MappedToRegionDistribution.txt")
y <- sort(counts$V1)

pdf("/user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405/03.MAPPING/MappedToRegionDistribution.pdf")
plot(y, pch=16)
dev.off()

#> summary(y)
#   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
#      1    5324   11170   14422   19892  135732 
    

In [ ]:
# Size distribution of the sequenced length (should be 190 bp but some are smaller or larger)
samtools view  CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4.bam| awk '{print $10}' |awk '{print length}'|sort | uniq -c

# 2) Reads 2: BC

## 2.0) Filter for sequencing quality



In [ ]:
# at least 80% of the NT (-u 20) must have a quality higher then 20 
# (same filtering as R1)
module load fastp/20180522-foss-2014a
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405

fastp --disable_adapter_trimming -q 20 -u 20 -i 00.RAW/CH1__4856bd__CheqSeq_OLS_Liesbeth_enhancer-barcode_S1_R2_001.fastq.gz -o 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20.fastq.gz

## 2.1) cut 5' adapter

In [ ]:
# cut 5' adaptor
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405
module load cutadapt/20180522-foss-2014a-Python-2.7.6
cutadapt -g CGAATTAATTCGGGCCCCGGTCC --trimmed-only \
--output 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20_trimmed5.fastq.gz \
02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20.fastq.gz

## 2.2) cut 3' adapter  + 17 bp long

In [ ]:
# cut 3' adaptor + demand it to be 17bp long
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405
cutadapt -a GATCGGCGCG --trimmed-only \
-m 17 -M 17 \
--output  02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20_trimmed5-3_BCs.fastq.gz \
 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20_trimmed5.fastq.gz

## 2.3) Filter more for sequencing quality

In [ ]:
# at least 90% of the NT (-u 10) must have a quality higher then 30 
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405
module load fastp/20180522-foss-2014a
fastp --disable_adapter_trimming -q 30 -u 10 -i 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20_trimmed5-3_BCs.fastq.gz -o 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20-Q30_trimmed5-3_BCs.fastq.gz

In [ ]:
# checked if all reads are still 17 bp
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405
zcat 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20-Q30_trimmed5-3_BCs.fastq.gz | sed -n '2~4p' | awk '{print length($0);}' | uniq -c

In [ ]:
# to check the distribution of the number of BCs
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405
zcat 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20-Q30_trimmed5-3_BCs.fastq.gz| sed -n '2~4p' | sort -k1nr | uniq -c | sort -k1nr | awk '{print $1 "\t" $2}'>  BC-distribution.txt

# 3) Link enhancers and BCs reads based on their read IDs
## 3.1) Reads with enhancer: 
- We will create 2 matrices:
    - From the enhancer bam (03.MAPPING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4.bam): ID - 'coord' - real sequence
    - From the BC fastq (02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20-Q30_trimmed5-3_BCs.fastq.gz): ID - BC
- Then we will merge the two matrices by read ID matrix with ID - BC - coord - real seq

In [ ]:
#  From the enhancer bam : get ID - 'coord' - real sequence
   # from 03.MAPPING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4.bam
   # ID: in first column
   # coord: in 3th column
   # seq: in 10th column
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405
mkdir 04.LINKING
module load SAMtools
samtools view 03.MAPPING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4.bam |\
awk '{print $1 "\t" $3 "\t" $10}' > 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4_Enhancers.matrix

In [ ]:
# From the BC fastq : get ID - BC
    # from 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20-Q30_trimmed5-3_BCs.fastq.gz
    # ID: in first of 4 lines ; first column
    # BC: in second of 4 lines

    # get the IDs (and remove @)
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405
zcat 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20-Q30_trimmed5-3_BCs.fastq.gz | sed -n '1~4p' |\
awk '{print $1}' | cut -c 2- \
> 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20-Q30_trimmed5-3_BCs.ID.matrix
    
    # get the BCs
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405
zcat 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20-Q30_trimmed5-3_BCs.fastq.gz | sed -n '2~4p' \
> 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20-Q30_trimmed5-3_BCs.BC.matrix

    # Column bind the IDs and BCs into 1 file
paste -d'\t' \
04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20-Q30_trimmed5-3_BCs.ID.matrix \
04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20-Q30_trimmed5-3_BCs.BC.matrix \
> 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20-Q30_trimmed5-3_BCs.ID-BC.matrix

In [ ]:
# Merge the enhancer and BC matrices based on their read IDs
    # Importantly: sort on the read IDs
    # ID - BC - coord - real seq
    # Join together based on read ID (-j 1 because in both files the ID is in the first column)
    # Specify the order of the columns you want to be printed
join -j 1 -o 1.1,1.2,2.2,2.3 \
<(sort -k1 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20-Q30_trimmed5-3_BCs.ID-BC.matrix) \
<(sort -k1 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4_Enhancers.matrix) \
| tr " " "\t" > 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4_Linked-Enhancers-BCs.matrix

## 3.2) Remove empty vector reads:
Similar as before: 
We will create 2 matrices:
       - From the empty vector fastq (02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5_emptyVector.fastq.gz): ID - remaining seq after clipping
       - From the BC fastq (02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20-Q30_trimmed5-3_BCs.fastq.gz): ID - BC 
            - already done: 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20-Q30_trimmed5-3_BCs.ID-BC.matrix

- Then we will merge the two matrices by read ID matrix with ID - BC - coord - real seq

- We will remove the BCs that are linked to empty enhancers from the enhancer-BC links


In [ ]:
# From the empty vector fastq : get ID - remaning seq after clipping
    # from 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5_emptyVector.fastq.gz
    # ID: in first of 4 lines ; first column
    # BC: in second of 4 lines

    # get the IDs (and remove @)
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405
zcat 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5_emptyVector.fastq.gz | sed -n '1~4p' |\
awk '{print $1}' | cut -c 2- \
> 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5_emptyVector.ID.matrix
    
    # get the BCs
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405
zcat 02.TRIMMED_FQ/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5_emptyVector.fastq.gz | sed -n '2~4p' \
> 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5_emptyVector.seq.matrix

    # Column bind the IDs and BCs into 1 file
paste -d'\t' \
04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5_emptyVector.ID.matrix \
04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5_emptyVector.seq.matrix \
> 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5_emptyVector.ID-seq.matrix

In [ ]:
# Merge the empty vector and BC matrices based on their read IDs
    # Importantly: sort on the read IDs
    # ID - BC - remaining seq
    # Join together based on read ID (-j 1 because in both files the ID is in the first column)
    # Specify the order of the columns you want to be printed
join -j 1 -o 1.1,1.2,2.2 \
<(sort -k1 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R2.Q20-Q30_trimmed5-3_BCs.ID-BC.matrix) \
<(sort -k1 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5_emptyVector.ID-seq.matrix) \
| tr " " "\t" > 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5_emptyVector_Linked-EmptyVector-BCs.matrix

In [ ]:
# How many of the BC linked to our enhancers are also linked to empty plasmids?
    # Sort based on the BC (-k2) before joining
    # We will join the empty vecotr - BC matrix with the enhancer BC matrix based on the BC (so -j 2)
    # print all the columns
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405

join -j 2 -o 1.1,1.2,1.3,1.4,2.1,2.2,2.3 \
<(sort -k2 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4_Linked-Enhancers-BCs.matrix) \
<(sort -k2 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5_emptyVector_Linked-EmptyVector-BCs.matrix) \
| tr " " "\t" > 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_EmptyVector-FullVector-BCs-overlap.matrix




__Remove empty vector BCs from the the enhancer-BC links__

In [ ]:
# BC in E-BC links also linekd to Empty vector:  04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_EmptyVector-FullVector-BCs-overlap.matrix
# -v Print a line for each unpairable line in file $number, instead of the normal output.
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405

join -j 2 -v 1 \
<(sort -k2 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4_Linked-Enhancers-BCs.matrix) \
<(sort -k2 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5_emptyVector_Linked-EmptyVector-BCs.matrix) \
| tr " " "\t" | awk '{print $2 "\t" $1 "\t" $3 "\t" $4}'>  04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4_Linked-Enhancers-BCs_woEmptyVector.matrix




__Reverse complement the BCs__

In [ ]:
# Reverse complement the BC
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405

cat 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4_Linked-Enhancers-BCs_woEmptyVector.matrix|awk '{print $2}' | rev | tr "ATGC" "TACG"\
> 04.LINKING/temp_BC-RC.txt
 
# Paste together the correct file
paste -d'\t' \
04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4_Linked-Enhancers-BCs_woEmptyVector.matrix \
04.LINKING/temp_BC-RC.txt | awk '{print $1 "\t" $5 "\t" $3 "\t" $4}'\
>04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4_Linked-Enhancers-BCs_woEmptyVector_BC-RC.matrix


## ==> Final file:  /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405/04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4_Linked-Enhancers-BCs_woEmptyVector_BC-RC.matrix



__Some stats__

In [ ]:
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405

# Number of unique BC
 cat  04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4_Linked-Enhancers-BCs_woEmptyVector.matrix | awk '{print $2}' | sort | uniq |wc -l

# Distribution of BC: so how many times each BC is appearing
cat  04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4_Linked-Enhancers-BCs_woEmptyVector.matrix | awk '{print $2}' |sort -k1nr | uniq -c | sort | awk '{print $1}' | uniq -c

# Number of unique designed regions found back
 cat  04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4_Linked-Enhancers-BCs_woEmptyVector.matrix | awk '{print $3}' | sort | uniq |wc -l
 
# By how many (non-uniq) BC is an enhancer represented?, so how many times appears each enhancer?
# Plot distribution

# Plot distribution how many reads map to which designed enhancers
cat 04.LINKING/CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4_Linked-Enhancers-BCs_woEmptyVector.matrix | awk '{print $3}' | sort  |uniq -c > 04.LINKING/Enhancer-BC-Distribution_woEmptyVector.txt

# R
module load R/3.4.1-foss-2014a-noX
module load HDF5/1.8.12-foss-2014a-serial
R
.libPaths("/user/leuven/317/vsc31703/stg_00002/R/x86_64-pc-linux-gnu-library/3.4")


counts <- read.table("/user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405/04.LINKING/Enhancer-BC-Distribution_woEmptyVector.txt")
y <- sort(counts$V1)

pdf("/user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405/04.LINKING/Enhancer-BC-Distribution_woEmptyVector.pdf")
plot(y, pch=16)
dev.off()

#> summary(y)
#   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
#    1.0   230.0   468.0   580.4   803.0  4887.0 
    
    
    

__Are BCs occuring multiple times linked to different regions?__

In [ ]:
cd /user/leuven/317/vsc31703/stg_00002/Melanoma-species/CHEQ-seq_synthetic/E-BC_coupling_NovaSeq6000_20190405/04.LINKING
## Quadriple
# Make file with BCs appearing 10 times
 cat  CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4_Linked-Enhancers-BCs_woEmptyVector.matrix | awk '{print $2}' | sort | uniq -c | sort -k1nr | awk '$1==10' |awk '{print $2}' > 10x_BCs.txt
 
 # Count for each quadriple BC to how many enhancer it is linked
touch 10x_BCs_Ecounts.txt
IFS=$'\n'     
set -f
for i in $(cat <  10x_BCs.txt); do
  cat CheqSeq_OLS_Liesbeth_enhancer-barcode_R1.Q20_trimmed5-3_full_Minimap.sorted.minQ4_Linked-Enhancers-BCs_woEmptyVector.matrix |awk '{print $1 "\t" $2 "\t" $3}' |\
  grep $i |awk '{print $3}' | sort | uniq |wc -l >>  10x_BCs_Ecounts.txt;
done


